# Feature Engineering, Feature / Permutation Importance, Pruning

## Setup

In [9]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import time, os, sys, warnings
import wandb
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import pickle
from sklearn.metrics import make_scorer
from sklearn.inspection import permutation_importance

from src.config import PROJECT_PATH, DATA_PATH, USE_WANDB, WANDB_PROJECT, WANDB_ENTITY
from src.tracking import ExperimentTracker
from src.training import train_and_evaluate_model, average_f1_score

SEED = 42

In [2]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000, "display.max_columns", 10000):
        display(df)

In [3]:
# Initialize the experiment tracker
tracker = ExperimentTracker(
    project_path=PROJECT_PATH,
    use_wandb=USE_WANDB,
    wandb_project_name=WANDB_PROJECT,
    wandb_entity=WANDB_ENTITY
)

In [4]:
# Load full training data, gesture mappings and inverse gesture mappings
df_train = pd.read_feather('processed_data/train_full.feather')

with open('processed_data/gesture_mappings.pkl', 'rb') as f:
    gesture_to_seq_type_map, gesture_map, inv_gesture_map = pickle.load(f)

In [5]:
display_all(df_train.head(2))

,row_id,sequence_type,sequence_id,sequence_counter,subject,orientation,behavior,phase,gesture,acc_x,acc_y,acc_z,rot_w,rot_x,rot_y,rot_z,thm_1,thm_2,thm_3,thm_4,thm_5,tof_1_v0,tof_1_v1,tof_1_v2,tof_1_v3,tof_1_v4,tof_1_v5,tof_1_v6,tof_1_v7,tof_1_v8,tof_1_v9,tof_1_v10,tof_1_v11,tof_1_v12,tof_1_v13,tof_1_v14,tof_1_v15,tof_1_v16,tof_1_v17,tof_1_v18,tof_1_v19,tof_1_v20,tof_1_v21,tof_1_v22,tof_1_v23,tof_1_v24,tof_1_v25,tof_1_v26,tof_1_v27,tof_1_v28,tof_1_v29,tof_1_v30,tof_1_v31,tof_1_v32,tof_1_v33,tof_1_v34,tof_1_v35,tof_1_v36,tof_1_v37,tof_1_v38,tof_1_v39,tof_1_v40,tof_1_v41,tof_1_v42,tof_1_v43,tof_1_v44,tof_1_v45,tof_1_v46,tof_1_v47,tof_1_v48,tof_1_v49,tof_1_v50,tof_1_v51,tof_1_v52,tof_1_v53,tof_1_v54,tof_1_v55,tof_1_v56,tof_1_v57,tof_1_v58,tof_1_v59,tof_1_v60,tof_1_v61,tof_1_v62,tof_1_v63,tof_2_v0,tof_2_v1,tof_2_v2,tof_2_v3,tof_2_v4,tof_2_v5,tof_2_v6,tof_2_v7,tof_2_v8,tof_2_v9,tof_2_v10,tof_2_v11,tof_2_v12,tof_2_v13,tof_2_v14,tof_2_v15,tof_2_v16,tof_2_v17,tof_2_v18,tof_2_v19,tof_2_v20,tof_2_v21,tof_2_v22,tof_2_v23,tof_2_v24,tof_2_v25,tof_2_v26,tof_2_v27,tof_2_v28,tof_2_v29,tof_2_v30,tof_2_v31,tof_2_v32,tof_2_v33,tof_2_v34,tof_2_v35,tof_2_v36,tof_2_v37,tof_2_v38,tof_2_v39,tof_2_v40,tof_2_v41,tof_2_v42,tof_2_v43,tof_2_v44,tof_2_v45,tof_2_v46,tof_2_v47,tof_2_v48,tof_2_v49,tof_2_v50,tof_2_v51,tof_2_v52,tof_2_v53,tof_2_v54,tof_2_v55,tof_2_v56,tof_2_v57,tof_2_v58,tof_2_v59,tof_2_v60,tof_2_v61,tof_2_v62,tof_2_v63,tof_3_v0,tof_3_v1,tof_3_v2,tof_3_v3,tof_3_v4,tof_3_v5,tof_3_v6,tof_3_v7,tof_3_v8,tof_3_v9,tof_3_v10,tof_3_v11,tof_3_v12,tof_3_v13,tof_3_v14,tof_3_v15,tof_3_v16,tof_3_v17,tof_3_v18,tof_3_v19,tof_3_v20,tof_3_v21,tof_3_v22,tof_3_v23,tof_3_v24,tof_3_v25,tof_3_v26,tof_3_v27,tof_3_v28,tof_3_v29,tof_3_v30,tof_3_v31,tof_3_v32,tof_3_v33,tof_3_v34,tof_3_v35,tof_3_v36,tof_3_v37,tof_3_v38,tof_3_v39,tof_3_v40,tof_3_v41,tof_3_v42,tof_3_v43,tof_3_v44,tof_3_v45,tof_3_v46,tof_3_v47,tof_3_v48,tof_3_v49,tof_3_v50,tof_3_v51,tof_3_v52,tof_3_v53,tof_3_v54,tof_3_v55,tof_3_v56,tof_3_v57,tof_3_v58,tof_3_v59,tof_3_v60,tof_3_v61,tof_3_v62,tof_3_v63,tof_4_v0,tof_4_v1,tof_4_v2,tof_4_v3,tof_4_v4,tof_4_v5,tof_4_v6,tof_4_v7,tof_4_v8,tof_4_v9,tof_4_v10,tof_4_v11,tof_4_v12,tof_4_v13,tof_4_v14,tof_4_v15,tof_4_v16,tof_4_v17,tof_4_v18,tof_4_v19,tof_4_v20,tof_4_v21,tof_4_v22,tof_4_v23,tof_4_v24,tof_4_v25,tof_4_v26,tof_4_v27,tof_4_v28,tof_4_v29,tof_4_v30,tof_4_v31,tof_4_v32,tof_4_v33,tof_4_v34,tof_4_v35,tof_4_v36,tof_4_v37,tof_4_v38,tof_4_v39,tof_4_v40,tof_4_v41,tof_4_v42,tof_4_v43,tof_4_v44,tof_4_v45,tof_4_v46,tof_4_v47,tof_4_v48,tof_4_v49,tof_4_v50,tof_4_v51,tof_4_v52,tof_4_v53,tof_4_v54,tof_4_v55,tof_4_v56,tof_4_v57,tof_4_v58,tof_4_v59,tof_4_v60,tof_4_v61,tof_4_v62,tof_4_v63,tof_5_v0,tof_5_v1,tof_5_v2,tof_5_v3,tof_5_v4,tof_5_v5,tof_5_v6,tof_5_v7,tof_5_v8,tof_5_v9,tof_5_v10,tof_5_v11,tof_5_v12,tof_5_v13,tof_5_v14,tof_5_v15,tof_5_v16,tof_5_v17,tof_5_v18,tof_5_v19,tof_5_v20,tof_5_v21,tof_5_v22,tof_5_v23,tof_5_v24,tof_5_v25,tof_5_v26,tof_5_v27,tof_5_v28,tof_5_v29,tof_5_v30,tof_5_v31,tof_5_v32,tof_5_v33,tof_5_v34,tof_5_v35,tof_5_v36,tof_5_v37,tof_5_v38,tof_5_v39,tof_5_v40,tof_5_v41,tof_5_v42,tof_5_v43,tof_5_v44,tof_5_v45,tof_5_v46,tof_5_v47,tof_5_v48,tof_5_v49,tof_5_v50,tof_5_v51,tof_5_v52,tof_5_v53,tof_5_v54,tof_5_v55,tof_5_v56,tof_5_v57,tof_5_v58,tof_5_v59,tof_5_v60,tof_5_v61,tof_5_v62,tof_5_v63,adult_child,age,sex,handedness,height_cm,shoulder_to_wrist_cm,elbow_to_wrist_cm
0,SEQ_000007_000000,Target,SEQ_000007,0,SUBJ_059520,Seated Lean Non Dom - FACE DOWN,Relaxes and moves hand to target location,Transition,Cheek - pinch skin,6.683594,6.214844,3.355469,0.134399,-0.355164,-0.447327,-0.809753,28.943842,31.822186,29.553024,28.592863,28.310535,131.0,134.0,132.0,135.0,98.0,74.0,64.0,60.0,-1.0,-1.0,152.0,153.0,141.0,89.0,68.0,63.0,-1.0,-1.0,-1.0,-1.0,169.0,118.0,86.0,73.0,-1.0,-1.0,-1.0,-1.0,-1.0,147.0,110.0,87.0,126.0,-1.0,-1.0,-1.0,-1.0,-1.0,137.0,108.0,115.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,128.0,110.0,129.0,140.0,-1.0,126.0,131.0,-1.0,-1.0,-1.0,108.0,122.0,139.0,113.0,12

In [6]:
print(f"Gesture Map: {gesture_map}")
print(f"\nInverted Gesture Map: {inv_gesture_map}")
print(f"\nGesture To Sequence Type: {gesture_to_seq_type_map}")

Gesture Map: {'Cheek - pinch skin': 0, 'Forehead - pull hairline': 1, 'Write name on leg': 2, 'Feel around in tray and pull out an object': 3, 'Neck - scratch': 4, 'Neck - pinch skin': 5, 'Eyelash - pull hair': 6, 'Eyebrow - pull hair': 7, 'Forehead - scratch': 8, 'Above ear - pull hair': 9, 'Wave hello': 10, 'Write name in air': 11, 'Text on phone': 12, 'Pull air toward your face': 13, 'Pinch knee/leg skin': 14, 'Scratch knee/leg skin': 15, 'Drink from bottle/cup': 16, 'Glasses on/off': 17}

Inverted Gesture Map: {0: 'Cheek - pinch skin', 1: 'Forehead - pull hairline', 2: 'Write name on leg', 3: 'Feel around in tray and pull out an object', 4: 'Neck - scratch', 5: 'Neck - pinch skin', 6: 'Eyelash - pull hair', 7: 'Eyebrow - pull hair', 8: 'Forehead - scratch', 9: 'Above ear - pull hair', 10: 'Wave hello', 11: 'Write name in air', 12: 'Text on phone', 13: 'Pull air toward your face', 14: 'Pinch knee/leg skin', 15: 'Scratch knee/leg skin', 16: 'Drink from bottle/cup', 17: 'Glasses on/of

In [7]:
N_SPLITS = 5
SEED = 42

MODEL_PARAMS = {
    'catboost': {
        'objective': 'MultiClass', # CatBoost specific
        'loss_function': 'MultiClass',
        'eval_metric': 'MultiClass',
        'iterations': 1000,
        'learning_rate': 0.05,
        'depth': 6,
        'l2_leaf_reg': 3.0,
        'random_seed': SEED,
        'verbose': False,
        'allow_writing_files': False,
        'task_type' : 'GPU',
    },
    'light_gbm': {
        'objective': 'multiclass',
        'num_class': 18, # Hard coding classes
        'metric': 'multi_logloss',
        'n_estimators': 1000,
        'learning_rate': 0.05,
        'num_leaves': 31,
        'lambda_l1': 0.1,
        'lambda_l2': 0.1,
        'bagging_fraction': 0.8,
        'feature_fraction': 0.8,
        'bagging_freq': 1,
        'random_state': SEED,
        'n_jobs': -1,
        'verbose': -1,
    }
}

## Feature Importance and Permutation Importance Functions

In [ ]:
def plot_feature_importance_builtin(model, feature_names, model_name="", top_n=100):
    "Plots built in feature importance"
    print(f"Plotting {model_name} Built-in Feature Importance...")
    if hasattr(model, 'feature_importances_'): # LGBM
        importances = model.feature_importances_ 
    elif hasattr(model, 'get_feature_importance'): #Catboost
        importances = model.get_feature_importances()
    else:
        print(f"Model {model_name} does not have a built-in feature importance method.")
        return None
    
    importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importances
    }).sort_values('importance', ascending=False).head(top_n)
    
    plt.figure(figsize=(10, max(6, top_n / 3)))
    sns.barplot(x='importance', y='feature', data=importance_df, palette="viridis")
    plt.title(f'Top {top_n} Features ({model_name} Built-in Importance)')
    plt.xlabel('Importance')
    plt.tight_layout()
    plt.show()
    
    return importance_df


def custom_permutation_importance_catboost(model, X_val, y_val, inv_map, seq_type_map, top_n=100, n_repeats=5, random_state=None):
    """
    Calculates permutation importance for a CatBoost model robustly.
    Operates directly on the pandas DataFrame to avoid sklearn's NumPy conversion issues.
    """
    print("Calculating Custom Permutation Importance for CatBoost (this may take a while)...")
    
    # 1. Define the correct scorer
    scorer = make_scorer(
        average_f1_score,
        needs_proba=False,
        inv_gesture_map=inv_map,
        gesture_to_seq_type_map=seq_type_map
    )

    # 2. Calculate baseline score using the ORIGINAL data
    y_pred_baseline = model.predict(X_val) 
    baseline_score = scorer._score_func(y_val, y_pred_baseline, inv_gesture_map=inv_map, gesture_to_seq_type_map=seq_type_map)
    print(f"  Baseline Score: {baseline_score:.4f}")

    # 3. Calculate importance for each feature
    importances = []
    importances_std = [] # To store std dev across repeats

    # Iterate through features (columns)
    for i, col in enumerate(X_val.columns):
        if i % 50 == 0: # Progress indicator for large feature sets
            print(f"  Processing feature {i+1}/{len(X_val.columns)}: {col}")
            
        scores = []
        # Repeat the permutation process
        for _ in range(n_repeats):
            # Shuffle the column
            X_permuted = X_val.copy() # Work on a copy
            X_permuted[col] = X_permuted[col].sample(frac=1, random_state=random_state).reset_index(drop=True)
            
            # PREDICT ON THE PERMUTED DATA
            y_pred_permuted = model.predict(X_permuted)
            # SCORE USING THE PERMUTED DATA AND PREDICTIONS
            permuted_score = scorer._score_func(y_val, y_pred_permuted, inv_gesture_map=inv_map, gesture_to_seq_type_map=seq_type_map)
            scores.append(permuted_score)
        
        # Calculate importance as the decrease in score
        # Importance = Baseline - Permutation Score
        importance_scores = [baseline_score - s for s in scores]
        mean_importance = np.mean(importance_scores)
        std_importance = np.std(importance_scores)
        
        importances.append(mean_importance)
        importances_std.append(std_importance)
        
        # Restore original data (implicit as we work on a copy each time)

    # 4. Create and Plot the Results
    perm_importance_df = pd.DataFrame({
        'feature': X_val.columns,
        'importance_mean': importances,
        'importance_std': importances_std,
    }).sort_values('importance_mean', ascending=False)

    # Filter for positive importances for plotting
    positive_importance_df = perm_importance_df[perm_importance_df['importance_mean'] > 0].head(top_n)
    
    if positive_importance_df.empty:
        print("\nNo features with positive permutation importance were found!!!")
    else:
        print("\nTop Features (Custom Permutation Importance - CatBoost):")
        print(positive_importance_df.head(10))
        
        plt.figure(figsize=(10, max(6, top_n / 3)))
        plt.barh(positive_importance_df['feature'], positive_importance_df['importance_mean'],
                 xerr=positive_importance_df['importance_std'], capsize=5, error_kw={'elinewidth': 0.5})
        plt.title(f'Top {top_n} Features (Custom Permutation Importance - CatBoost)')
        plt.xlabel('Importance (Decrease in F1 Score)')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.show()
        
    return perm_importance_df


def calculate_permutation_importance_sklearn(model, X_val, y_val, inv_map, seq_type_map, top_n=100, n_repeats=5, random_state=None):
    """
    Calculates permutation importance using sklearn's function. Suitable for LightGBM.
    """
    print("Calculating Permutation Importance using sklearn (for LightGBM)...")
    
    scorer = make_scorer(
        average_f1_score,
        needs_proba=False,
        inv_gesture_map=inv_map,
        gesture_to_seq_type_map=seq_type_map
    )
    
    # Calculate permutation importance using sklearn
    try:
        perm_result = permutation_importance(
            model, X_val, y_val,
            n_repeats=n_repeats,
            random_state=random_state,
            n_jobs=-1,
            scoring=scorer
        )
    except Exception as e:
        print(f"Error during sklearn permutation importance calculation: {e}")
        return None
    
    # Create dataframe and plot
    perm_importance_df = pd.DataFrame({
        'feature': X_val.columns,
        'importance_mean': perm_result.importances_mean,
        'importance_std': perm_result.importances_std,
    }).sort_values('importances_mean', ascending=False)
    
    # Filter for positive importances for plotting
    positive_importance_df = perm_importance_df[perm_importance_df['importance_mean'] > 0].head(top_n)
    
    if positive_importance_df.empty:
        print("\nNo features with positive permutation importance were found!!!")
    else:
        print("\nTop Features (Permutation Importance - sklearn - LightGBM):")
        print(positive_importance_df.head(10))
        
        plt.figure(figsize=(10, max(6, top_n / 3)))
        plt.barh(positive_importance_df['feature'], positive_importance_df['importance_mean'],
                 xerr=positive_importance_df['importance_std'], capsize=5, error_kw={'elinewidth': 0.5})
        plt.title(f'Top {top_n} Features (Permutation Importance - sklearn - LightGBM)')
        plt.xlabel('Importance (Decrease in F1 Score)')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.show()
        
    return perm_importance_df

In [ ]:
def analyze_model_importance(model, model_name, X_train, y_train, X_val, y_val, inv_map, seq_type_map, top_n=30):
    """
    Orchestrates the training and importance analysis for a given model.
    """
    print(f"\n--- Analyzing Feature Importance for {model_name} ---")
    
    # --- 1. Train the Model (Assuming a generic training function exists) ---
    # You would call your specific training function here, e.g., train_catboost or train_lightgbm
    # model = train_your_model_function(X_train, y_train, X_val, y_val, model_type=model_name)
    # For this example, we assume 'model' is already trained and passed in.
    # Ensure the model is fitted before proceeding.
    if not hasattr(model, 'predict'):
        raise ValueError(f"Provided model for {model_name} does not seem to be fitted or is invalid.")

    # --- 2. Plot Built-in Importance ---
    builtin_importance_df = plot_feature_importance_builtin(model, X_train.columns, model_name=model_name, top_n=top_n)
    
    # --- 3. Calculate and Plot Permutation Importance ---
    if model_name.lower() == "catboost":
        # Use the custom function for CatBoost
        perm_importance_df = custom_permutation_importance_catboost(
            model, X_val, y_val, inv_map, seq_type_map, top_n=top_n, n_repeats=5, random_state=SEED
        )
    elif model_name.lower() in ["lightgbm", "lgbm"]:
        # Use sklearn's function for LightGBM
        perm_importance_df = calculate_permutation_importance_sklearn(
            model, X_val, y_val, inv_map, seq_type_map, top_n=top_n, n_repeats=5, random_state=SEED
        )
    else:
        print(f"Warning: Permutation importance function not defined for model type '{model_name}'. Skipping.")
        perm_importance_df = None

    print(f"--- Analysis Complete for {model_name} ---\n")
    return builtin_importance_df, perm_importance_df